In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
from warnings import warn
from IPython.core.display import clear_output

from time import time
from datetime import datetime


In [2]:
url = 'https://www.covers.com/sports/NBA/matchups?selectedDate=2020-03-11'

response = get(url)
print(response.text[:500])




<!DOCTYPE html>
<html>
<head>
	
	


    <title>NBA Matchups 2019-2020 - Basketball Previews, Scores, Lines &amp; Odds</title>
    <meta name="description" content="Find the top 2019-2020 NBA Matchups betting previews &amp; odds for opening game odds, closing lines, betting trends, ATS results and more for pro basketball.">
        <meta name="keywords" content="nba, nba scores, nba matchups, basketball, nba games, nba results, nba logros , escore nba, resultados nba">
        <meta


In [3]:
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [4]:
game_containers = html_soup.find_all('div', class_ = 'cmg_matchup_game_box cmg_game_data')
print(type(game_containers))
print(len(game_containers))


<class 'bs4.element.ResultSet'>
4


In [5]:
games = html_soup.find_all('tbody')
for game in games:
    print(game.text.split())
    

['DET', '29', '26', '29', '22', '106', 'PHI', '34', '36', '29', '25', '124']
['CHA', '22', '37', '28', '22', '109', 'MIA', '40', '11', '26', '21', '98']
['NY', '33', '34', '29', '22', '18', '136', 'ATL', '24', '26', '28', '40', '13', '131']
['DEN', '29', '21', '35', '12', '97', 'DAL', '33', '24', '27', '29', '113']


In [6]:
first = games[0]


In [7]:
#Extract Quarter Scores/Totals
first_game = game_containers[0]
scores = first_game.find('div', class_ = 'cmg_matchup_line_score')
scores = scores.find_all('tr')

q1_pts_away = 0 
q1_pts_home = 0
q2_pts_away = 0
q2_pts_home = 0
q3_pts_away = 0
q3_pts_home = 0
q4_pts_away = 0
q4_pts_home = 0
ot1_pts_away = 0
ot1_pts_home = 0
ot2_pts_away = 0
ot2_pts_home = 0
ot3_pts_away = 0
ot3_pts_home = 0
ot4_pts_away = 0
ot4_pts_home = 0

quarters_away = [q1_pts_away, q2_pts_away, q3_pts_away, q4_pts_away, ot1_pts_away, ot2_pts_away, ot3_pts_away, ot4_pts_away]
quarters_home = [q1_pts_home, q2_pts_home, q3_pts_home, q4_pts_home, ot1_pts_home, ot2_pts_home, ot3_pts_home, ot4_pts_home]

scores_list_away = scores[1].text.split()[1:-1]
for i in range(len(scores_list_away)):
    quarters_away[i] = scores_list_away[i]

scores_list_home = scores[2].text.split()[1:-1]
for i in range(len(scores_list_home)):
    quarters_home[i] = scores_list_home[i]
    
    
q1_pts_away, q2_pts_away, q3_pts_away, q4_pts_away, ot1_pts_away, ot2_pts_away, ot3_pts_away, ot4_pts_away = tuple(quarters_away)
q1_pts_home, q2_pts_home, q3_pts_home, q4_pts_home, ot1_pts_home, ot2_pts_home, ot3_pts_home, ot4_pts_home = tuple(quarters_home)



In [8]:
# Extract spreads (for home team) and over/unders
game_data = html_soup.find_all('div', class_ = 'cmg_game_data')
first_bet = game_data[0]

away_team_abbr = first_bet['data-away-team-shortname-search']
home_team_abbr = first_bet['data-home-team-shortname-search']
event_id = first_bet['data-event-id']
home_score = first_bet['data-home-score']
away_score = first_bet['data-away-score']
spread = first_bet['data-game-odd']
over_under = first_bet['data-game-total']

print(away_team_abbr, home_team_abbr, event_id, home_score, away_score, spread, over_under)

DET PHI 1007658 124 106 -12 209.5


## Putting it together - Scraping the Whole Page

In [9]:
# intialize lists to store data

home_team_abbr = []
away_team_abbr = []
event_id = []
home_score = []
away_score = []
spread = []
over_under = []
q1_pts_away = [] 
q1_pts_home = []
q2_pts_away = []
q2_pts_home = []
q3_pts_away = []
q3_pts_home = []
q4_pts_away = []
q4_pts_home = []
ot1_pts_away = []
ot1_pts_home = []
ot2_pts_away = []
ot2_pts_home = []
ot3_pts_away = []
ot3_pts_home = []
ot4_pts_away = []
ot4_pts_home = []
date = []

In [10]:
type(game_containers[3].find('td', class_='cmg_matchup_line_score_total'))

bs4.element.Tag

In [11]:
for game in game_containers:
    if game.find('td', class_ ='cmg_matchup_line_score_total') is not None:
        quarter_scores = game.find('div', class_ = 'cmg_matchup_line_score')
        quarter_scores = quarter_scores.find_all('tr')
        
        # initialize these variables to 0 so that I can store them as a list (there is probably a better way to do this)
        q1_pts_a = 0 
        q1_pts_h = 0
        q2_pts_a = 0
        q2_pts_h = 0
        q3_pts_a = 0
        q3_pts_h = 0
        q4_pts_a = 0
        q4_pts_h = 0
        ot1_pts_a = 0
        ot1_pts_h = 0
        ot2_pts_a = 0
        ot2_pts_h = 0
        ot3_pts_a = 0
        ot3_pts_h = 0
        ot4_pts_a = 0
        ot4_pts_h = 0

        quarters_away = [q1_pts_a, q2_pts_a, q3_pts_a, q4_pts_a, ot1_pts_a, ot2_pts_a, ot3_pts_a, ot4_pts_a]
        quarters_home = [q1_pts_h, q2_pts_h, q3_pts_h, q4_pts_h, ot1_pts_h, ot2_pts_h, ot3_pts_h, ot4_pts_h]

        scores_list_away = quarter_scores[1].text.split()[1:-1]     # this removes the team_name and the final score, since I already have those and just leaves the quarter scores
        
        for i in range(len(scores_list_away)):
            quarters_away[i] = scores_list_away[i]

        scores_list_home = quarter_scores[2].text.split()[1:-1]
        for i in range(len(scores_list_home)):
            quarters_home[i] = scores_list_home[i]

        # store them into these variables
        q1_pts_a, q2_pts_a, q3_pts_a, q4_pts_a, ot1_pts_a, ot2_pts_a, ot3_pts_a, ot4_pts_a = tuple(quarters_away)
        q1_pts_h, q2_pts_h, q3_pts_h, q4_pts_h, ot1_pts_h, ot2_pts_h, ot3_pts_h, ot4_pts_h = tuple(quarters_home)
        
        q1_pts_away.append(q1_pts_a)
        q2_pts_away.append(q2_pts_a)
        q3_pts_away.append(q3_pts_a)
        q4_pts_away.append(q4_pts_a)
        ot1_pts_away.append(ot1_pts_a)
        ot2_pts_away.append(ot2_pts_a)
        ot3_pts_away.append(ot3_pts_a)
        ot4_pts_away.append(ot4_pts_a)
        
        q1_pts_home.append(q1_pts_h)
        q2_pts_home.append(q2_pts_h)
        q3_pts_home.append(q3_pts_h)
        q4_pts_home.append(q4_pts_h)
        ot1_pts_home.append(ot1_pts_h)
        ot2_pts_home.append(ot2_pts_h)
        ot3_pts_home.append(ot3_pts_h)
        ot4_pts_home.append(ot4_pts_h)
  
        away_team_abbr.append(game['data-away-team-shortname-search'])
        home_team_abbr.append(game['data-home-team-shortname-search'])
        event_id.append(game['data-event-id'])
        home_score.append(game['data-home-score'])
        away_score.append(game['data-away-score'])
        spread.append(game['data-game-odd'])
        over_under.append(game['data-game-total'])
        date.append(game['data-game-date'])
        
        


In [12]:
len(spread)


4

In [15]:
betting_data = pd.DataFrame({'event_id':event_id, 
                             'date':date,
                             'home_team_abbr':home_team_abbr,
                             'away_team_abbr':away_team_abbr, 
                             'home_score' : home_score,
                             'away_score' : away_score,
                             'q1_pts_home': q1_pts_home,
                             'q2_pts_home': q2_pts_home,
                             'q3_pts_home': q3_pts_home,
                             'q4_pts_home': q4_pts_home,
                             'ot1_pts_home': ot1_pts_home,
                             'ot2_pts_home': ot2_pts_home,
                             'ot3_pts_home': ot3_pts_home,
                             'ot4_pts_home': ot4_pts_home,
                             'q1_pts_away': q1_pts_home,
                             'q2_pts_away': q2_pts_home,
                             'q3_pts_away': q3_pts_home,
                             'q4_pts_away': q4_pts_home,
                             'ot1_pts_away': ot1_pts_home,
                             'ot2_pts_away': ot2_pts_home,
                             'ot3_pts_away': ot3_pts_home,
                             'ot4_pts_away': ot4_pts_home,
                             'spread' : spread,
                             'total' : over_under
                            })



In [16]:
betting_data

,event_id,date,home_team_abbr,away_team_abbr,home_score,away_score,q1_pts_home,q2_pts_home,q3_pts_home,q4_pts_home,...,q1_pts_away,q2_pts_away,q3_pts_away,q4_pts_away,ot1_pts_away,ot2_pts_away,ot3_pts_away,ot4_pts_away,spread,total
0,1007658,2020-03-11 19:00:00,PHI,DET,124,106,34,36,29,25,...,34,36,29,25,0,0,0,0,-12,209.5
1,1007657,2020-03-11 19:30:00,MIA,CHA,98,109,40,11,26,21,...,40,11,26,21,0,0,0,0,-11,206.5
2,1007656,2020-03-11 19:30:00,ATL,NY,131,136,24,26,28,40,...,24,26,28,40,13,0,0,0,-5,232.5
3,1007660,2020-03-11 20:00:00,DAL,DEN,113,97,33,24,27,29,...,33,24,27,29,0,0,0,0,2.5,218


# Scraping Multiple Pages

In [2]:
from warnings import warn
warn("Warning Simulation")
from IPython.core.display import clear_output
from time import time
start_time = time()

requests = 0
import numpy as np
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.library.parameters import SeasonAll

C:\Users\Jordan Nishimura\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Warning Simulation
  


In [3]:
seasons = ['20{}-{}'.format(x, x+1) for x in range(13, 15)]
dates = np.array([])
for season in seasons:
    games = leaguegamelog.LeagueGameLog(season=season).get_data_frames()[0]
    dates = np.append(dates, games['GAME_DATE'].unique())

print(len(dates))
print(dates[0], dates[-1])


325
2013-10-29 2015-04-15


In [4]:
dates = dates.tolist()

In [10]:
# intialize lists to store data

home_team_abbr = []
away_team_abbr = []
event_id = []
home_score = []
away_score = []
spread = []
total = []
q1_pts_away = [] 
q1_pts_home = []
q2_pts_away = []
q2_pts_home = []
q3_pts_away = []
q3_pts_home = []
q4_pts_away = []
q4_pts_home = []
ot1_pts_away = []
ot1_pts_home = []
ot2_pts_away = []
ot2_pts_home = []
ot3_pts_away = []
ot3_pts_home = []
ot4_pts_away = []
ot4_pts_home = []
game_date = []

# To help control loop rate
start_time = time()
requests = 0

for date in dates:
    url = 'https://www.covers.com/sports/NBA/matchups?selectedDate={}'.format(date)        #url for every day in the 2015-2020 season (there will be repeats on days that have no game. we can delete these after)
    # Make a request
    response = get(url)
    
    # Pause the Loop
    sleep(randint(2, 4))
    
    #Monitor the requests
    requests+=1
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {}'.format(requests, requests/elapsed_time))
    clear_output(wait=True)
    
    # Warning for non-200 status codes
    if response.status_code != 200:
        warn('request: {}; Status code: {}'.format(requests, response.status_code))
        
    # Break loop if number of requests is greater than expetcted
#     if requests > 300:
#         warn('Number of requests was greater than expected')
#         break
    
    html_soup = BeautifulSoup(response.text, 'html.parser')
    
    game_containers = html_soup.find_all('div', class_ = 'cmg_matchup_game_box cmg_game_data')

    for game in game_containers:
        if game.find('td', class_ ='cmg_matchup_line_score_total') is not None:
            quarter_scores = game.find('div', class_ = 'cmg_matchup_line_score')
            quarter_scores = quarter_scores.find_all('tr')

            # initialize these variables to 0 so that I can store them as a list (there is probably a better way to do this)
            q1_pts_a = 0 
            q1_pts_h = 0
            q2_pts_a = 0
            q2_pts_h = 0
            q3_pts_a = 0
            q3_pts_h = 0
            q4_pts_a = 0
            q4_pts_h = 0
            ot1_pts_a = 0
            ot1_pts_h = 0
            ot2_pts_a = 0
            ot2_pts_h = 0
            ot3_pts_a = 0
            ot3_pts_h = 0
            ot4_pts_a = 0
            ot4_pts_h = 0

            quarters_away = [q1_pts_a, q2_pts_a, q3_pts_a, q4_pts_a, ot1_pts_a, ot2_pts_a, ot3_pts_a, ot4_pts_a]
            quarters_home = [q1_pts_h, q2_pts_h, q3_pts_h, q4_pts_h, ot1_pts_h, ot2_pts_h, ot3_pts_h, ot4_pts_h]
            
            # this removes the team_name and the final score, since I already have those and just leaves the quarter scores
            scores_list_away = quarter_scores[1].text.split()[1:-1]     

            for i in range(len(scores_list_away)):
                quarters_away[i] = scores_list_away[i]

            scores_list_home = quarter_scores[2].text.split()[1:-1]
            for i in range(len(scores_list_home)):
                quarters_home[i] = scores_list_home[i]

            # store them into these variables
            q1_pts_a, q2_pts_a, q3_pts_a, q4_pts_a, ot1_pts_a, ot2_pts_a, ot3_pts_a, ot4_pts_a = tuple(quarters_away)
            q1_pts_h, q2_pts_h, q3_pts_h, q4_pts_h, ot1_pts_h, ot2_pts_h, ot3_pts_h, ot4_pts_h = tuple(quarters_home)

            q1_pts_away.append(q1_pts_a)
            q2_pts_away.append(q2_pts_a)
            q3_pts_away.append(q3_pts_a)
            q4_pts_away.append(q4_pts_a)
            ot1_pts_away.append(ot1_pts_a)
            ot2_pts_away.append(ot2_pts_a)
            ot3_pts_away.append(ot3_pts_a)
            ot4_pts_away.append(ot4_pts_a)

            q1_pts_home.append(q1_pts_h)
            q2_pts_home.append(q2_pts_h)
            q3_pts_home.append(q3_pts_h)
            q4_pts_home.append(q4_pts_h)
            ot1_pts_home.append(ot1_pts_h)
            ot2_pts_home.append(ot2_pts_h)
            ot3_pts_home.append(ot3_pts_h)
            ot4_pts_home.append(ot4_pts_h)

            away_team_abbr.append(game['data-away-team-shortname-search'])
            home_team_abbr.append(game['data-home-team-shortname-search'])
            event_id.append(game['data-event-id'])
            home_score.append(game['data-home-score'])
            away_score.append(game['data-away-score'])
            spread.append(game['data-game-odd'])
            total.append(game['data-game-total'])
            game_date.append(date)

Request: 325; Frequency: 0.2623477482921404


In [11]:
nba_betting_data_2013_2014 = pd.DataFrame({'event_id':event_id, 
                             'date':game_date,
                             'home_team_abbr':home_team_abbr,
                             'away_team_abbr':away_team_abbr, 
                             'home_score' : home_score,
                             'away_score' : away_score,
                             'q1_pts_home': q1_pts_home,
                             'q2_pts_home': q2_pts_home,
                             'q3_pts_home': q3_pts_home,
                             'q4_pts_home': q4_pts_home,
                             'ot1_pts_home': ot1_pts_home,
                             'ot2_pts_home': ot2_pts_home,
                             'ot3_pts_home': ot3_pts_home,
                             'ot4_pts_home': ot4_pts_home,
                             'q1_pts_away': q1_pts_home,
                             'q2_pts_away': q2_pts_home,
                             'q3_pts_away': q3_pts_home,
                             'q4_pts_away': q4_pts_home,
                             'ot1_pts_away': ot1_pts_home,
                             'ot2_pts_away': ot2_pts_home,
                             'ot3_pts_away': ot3_pts_home,
                             'ot4_pts_away': ot4_pts_home,
                             'spread' : spread,
                             'total' : total
                            })



In [12]:
nba_betting_data_2013_2014

,event_id,date,home_team_abbr,away_team_abbr,home_score,away_score,q1_pts_home,q2_pts_home,q3_pts_home,q4_pts_home,...,q1_pts_away,q2_pts_away,q3_pts_away,q4_pts_away,ot1_pts_away,ot2_pts_away,ot3_pts_away,ot4_pts_away,spread,total
0,864184,2013-10-29,IND,ORL,97,87,23,17,29,28,...,23,17,29,28,0,0,0,0,-12,189.5
1,864185,2013-10-29,MIA,CHI,107,95,17,37,24,29,...,17,37,24,29,0,0,0,0,-5,188.5
2,864186,2013-10-29,LAL,LAC,116,103,28,27,20,41,...,28,27,20,41,0,0,0,0,9.5,202.5
3,864188,2013-10-30,TOR,BOS,93,87,21,28,22,22,...,21,28,22,22,0,0,0,0,-8,189.5
4,864200,2013-10-30,CLE,BK,98,94,27,22,30,19,...,27,22,30,19,0,0,0,0,3,194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,883922,2015-04-15,NY,DET,90,112,25,21,24,20,...,25,21,24,20,0,0,0,0,9,193
2456,883920,2015-04-15,BK,ORL,101,88,23,25,27,26,...,23,25,27,26,0,0,0,0,-9,203.5
2457,883916,2015-04-15,MEM,IND,95,83,22,32,19,22,...,22,32,19,22,0,0,0,0,2.5,184.5
2458,883925,2015-04-15,GS,DEN,133,126,35,36,29,33,...,35,36,29,33,0,0,0,0,-11.5,215.5


In [13]:
nba_betting_data_2015_2020 = pd.read_csv('./data/nba_betting_data_2015_2020.csv')
nba_betting_data_2015_2020

,Unnamed: 0,event_id,date,home_team_abbr,away_team_abbr,home_score,away_score,q1_pts_home,q2_pts_home,q3_pts_home,...,q1_pts_away,q2_pts_away,q3_pts_away,q4_pts_away,ot1_pts_away,ot2_pts_away,ot3_pts_away,ot4_pts_away,spread,total
0,0,896350,2015-10-27,ATL,DET,94,106,34,36,29,...,34,36,29,25,0,0,0,0,-7.0,196.5
1,1,896351,2015-10-27,CHI,CLE,97,95,34,36,29,...,34,36,29,25,0,0,0,0,-4.0,198.5
2,2,896352,2015-10-27,GS,NO,111,95,34,36,29,...,34,36,29,25,0,0,0,0,-10.5,215.0
3,3,896353,2015-10-28,ORL,WAS,87,88,34,36,29,...,34,36,29,25,0,0,0,0,4.5,203.0
4,4,896356,2015-10-28,DET,UTA,92,87,34,36,29,...,34,36,29,25,0,0,0,0,0.0,191.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6227,3948,1030404,2021-01-25,CLE,LAL,108,115,34,36,29,...,34,36,29,25,0,0,0,0,12.0,214.5
6228,3949,1030416,2021-01-25,DAL,DEN,113,117,34,36,29,...,34,36,29,25,0,0,0,0,2.0,223.0
6229,3950,1030407,2021-01-25,CHI,BOS,103,119,34,36,29,...,34,36,29,25,0,0,0,0,3.5,227.5
6230,3951,1030421,2021-01-25,POR,OKC,122,125,34,36,29,...,34,36,29,25,0,0,0,0,-4.5,226.5


In [18]:
betting_data_2013_2020 = pd.concat([nba_betting_data_2013_2014, nba_betting_data_2015_2020])
betting_data_2013_2020 = betting_data_2013_2020.drop(columns=['Unnamed: 0'])
betting_data_2013_2020.to_csv('./data/nba_betting_data_2013_2020.csv', index=False)

In [19]:
betting_data_2013_2020

,event_id,date,home_team_abbr,away_team_abbr,home_score,away_score,q1_pts_home,q2_pts_home,q3_pts_home,q4_pts_home,...,q1_pts_away,q2_pts_away,q3_pts_away,q4_pts_away,ot1_pts_away,ot2_pts_away,ot3_pts_away,ot4_pts_away,spread,total
0,864184,2013-10-29,IND,ORL,97,87,23,17,29,28,...,23,17,29,28,0,0,0,0,-12,189.5
1,864185,2013-10-29,MIA,CHI,107,95,17,37,24,29,...,17,37,24,29,0,0,0,0,-5,188.5
2,864186,2013-10-29,LAL,LAC,116,103,28,27,20,41,...,28,27,20,41,0,0,0,0,9.5,202.5
3,864188,2013-10-30,TOR,BOS,93,87,21,28,22,22,...,21,28,22,22,0,0,0,0,-8,189.5
4,864200,2013-10-30,CLE,BK,98,94,27,22,30,19,...,27,22,30,19,0,0,0,0,3,194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6227,1030404,2021-01-25,CLE,LAL,108,115,34,36,29,25,...,34,36,29,25,0,0,0,0,12,214.5
6228,1030416,2021-01-25,DAL,DEN,113,117,34,36,29,25,...,34,36,29,25,0,0,0,0,2,223
6229,1030407,2021-01-25,CHI,BOS,103,119,34,36,29,25,...,34,36,29,25,0,0,0,0,3.5,227.5
6230,1030421,2021-01-25,POR,OKC,122,125,34,36,29,25,...,34,36,29,25,0,0,0,0,-4.5,226.5


In [21]:
logs= leaguegamelog.LeagueGameLog(season='2020-21').get_data_frames()[0]
logs['GAME_DATE'].unique()

array(['2020-12-22', '2020-12-23', '2020-12-25', '2020-12-26',
       '2020-12-27', '2020-12-28', '2020-12-29', '2020-12-30',
       '2020-12-31', '2021-01-01', '2021-01-02', '2021-01-03',
       '2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07',
       '2021-01-08', '2021-01-09', '2021-01-10', '2021-01-11',
       '2021-01-12', '2021-01-13', '2021-01-14', '2021-01-15',
       '2021-01-16', '2021-01-17', '2021-01-18', '2021-01-19',
       '2021-01-20', '2021-01-21', '2021-01-22', '2021-01-23',
       '2021-01-24', '2021-01-25', '2021-01-26', '2021-01-27',
       '2021-01-28', '2021-01-29', '2021-01-30', '2021-01-31',
       '2021-02-01', '2021-02-02', '2021-02-03', '2021-02-04'],
      dtype=object)

In [24]:
current_data = pd.read_csv('./data/nba_betting_data_2013_2020.csv')
current_dates = current_data['date'].unique()
current_dates


array(['2013-10-29', '2013-10-30', '2013-10-31', ..., '2021-01-23',
       '2021-01-24', '2021-01-25'], dtype=object)

In [27]:
def update_betting_data():
    my_data = pd.read_csv('./data/nba_betting_data_2013_present.csv')
    my_dates = my_data['date'].unique()
    
    full_logs = leaguegamelog.LeagueGameLog(season='2020-21').get_data_frames()[0]
    full_dates = full_logs['GAME_DATE'].unique()
    
    missing_dates = set(full_dates) - set(my_dates)
    
    num_missing_dates = len(missing_dates)
    first_missing_date = list(missing_dates)[0]
    
    home_team_abbr = []
    away_team_abbr = []
    event_id = []
    home_score = []
    away_score = []
    spread = []
    total = []
    q1_pts_away = [] 
    q1_pts_home = []
    q2_pts_away = []
    q2_pts_home = []
    q3_pts_away = []
    q3_pts_home = []
    q4_pts_away = []
    q4_pts_home = []
    ot1_pts_away = []
    ot1_pts_home = []
    ot2_pts_away = []
    ot2_pts_home = []
    ot3_pts_away = []
    ot3_pts_home = []
    ot4_pts_away = []
    ot4_pts_home = []
    game_date = []

    # To help control loop rate
    start_time = time()
    requests = 0

    for date in missing_dates:
        url = 'https://www.covers.com/sports/NBA/matchups?selectedDate={}'.format(date)       
        # Make a request
        response = get(url)

        # Pause the Loop
        sleep(randint(2, 4))

        #Monitor the requests
        requests+=1
        elapsed_time = time() - start_time
        print('Request: {}; Frequency: {}'.format(requests, requests/elapsed_time))
        clear_output(wait=True)

        # Warning for non-200 status codes
        if response.status_code != 200:
            warn('request: {}; Status code: {}'.format(requests, response.status_code))

        # Break loop if number of requests is greater than expetcted
    #     if requests > 300:
    #         warn('Number of requests was greater than expected')
    #         break

        html_soup = BeautifulSoup(response.text, 'html.parser')

        game_containers = html_soup.find_all('div', class_ = 'cmg_matchup_game_box cmg_game_data')

        for game in game_containers:
            if game.find('td', class_ ='cmg_matchup_line_score_total') is not None:
                quarter_scores = game.find('div', class_ = 'cmg_matchup_line_score')
                quarter_scores = quarter_scores.find_all('tr')

                # initialize these variables to 0 so that I can store them as a list (there is probably a better way to do this)
                q1_pts_a = 0 
                q1_pts_h = 0
                q2_pts_a = 0
                q2_pts_h = 0
                q3_pts_a = 0
                q3_pts_h = 0
                q4_pts_a = 0
                q4_pts_h = 0
                ot1_pts_a = 0
                ot1_pts_h = 0
                ot2_pts_a = 0
                ot2_pts_h = 0
                ot3_pts_a = 0
                ot3_pts_h = 0
                ot4_pts_a = 0
                ot4_pts_h = 0

                quarters_away = [q1_pts_a, q2_pts_a, q3_pts_a, q4_pts_a, ot1_pts_a, ot2_pts_a, ot3_pts_a, ot4_pts_a]
                quarters_home = [q1_pts_h, q2_pts_h, q3_pts_h, q4_pts_h, ot1_pts_h, ot2_pts_h, ot3_pts_h, ot4_pts_h]

                # this removes the team_name and the final score, since I already have those and just leaves the quarter scores
                scores_list_away = quarter_scores[1].text.split()[1:-1]     

                for i in range(len(scores_list_away)):
                    quarters_away[i] = scores_list_away[i]

                scores_list_home = quarter_scores[2].text.split()[1:-1]
                for i in range(len(scores_list_home)):
                    quarters_home[i] = scores_list_home[i]

                # store them into these variables
                q1_pts_a, q2_pts_a, q3_pts_a, q4_pts_a, ot1_pts_a, ot2_pts_a, ot3_pts_a, ot4_pts_a = tuple(quarters_away)
                q1_pts_h, q2_pts_h, q3_pts_h, q4_pts_h, ot1_pts_h, ot2_pts_h, ot3_pts_h, ot4_pts_h = tuple(quarters_home)

                q1_pts_away.append(q1_pts_a)
                q2_pts_away.append(q2_pts_a)
                q3_pts_away.append(q3_pts_a)
                q4_pts_away.append(q4_pts_a)
                ot1_pts_away.append(ot1_pts_a)
                ot2_pts_away.append(ot2_pts_a)
                ot3_pts_away.append(ot3_pts_a)
                ot4_pts_away.append(ot4_pts_a)

                q1_pts_home.append(q1_pts_h)
                q2_pts_home.append(q2_pts_h)
                q3_pts_home.append(q3_pts_h)
                q4_pts_home.append(q4_pts_h)
                ot1_pts_home.append(ot1_pts_h)
                ot2_pts_home.append(ot2_pts_h)
                ot3_pts_home.append(ot3_pts_h)
                ot4_pts_home.append(ot4_pts_h)

                away_team_abbr.append(game['data-away-team-shortname-search'])
                home_team_abbr.append(game['data-home-team-shortname-search'])
                event_id.append(game['data-event-id'])
                home_score.append(game['data-home-score'])
                away_score.append(game['data-away-score'])
                spread.append(game['data-game-odd'])
                total.append(game['data-game-total'])
                game_date.append(date)
                
    print("updated {} dates starting from {}".format(num_missing_dates, first_missing_date))
    
    missing_data = pd.DataFrame({'event_id':event_id, 
                             'date':game_date,
                             'home_team_abbr':home_team_abbr,
                             'away_team_abbr':away_team_abbr, 
                             'home_score' : home_score,
                             'away_score' : away_score,
                             'q1_pts_home': q1_pts_home,
                             'q2_pts_home': q2_pts_home,
                             'q3_pts_home': q3_pts_home,
                             'q4_pts_home': q4_pts_home,
                             'ot1_pts_home': ot1_pts_home,
                             'ot2_pts_home': ot2_pts_home,
                             'ot3_pts_home': ot3_pts_home,
                             'ot4_pts_home': ot4_pts_home,
                             'q1_pts_away': q1_pts_home,
                             'q2_pts_away': q2_pts_home,
                             'q3_pts_away': q3_pts_home,
                             'q4_pts_away': q4_pts_home,
                             'ot1_pts_away': ot1_pts_home,
                             'ot2_pts_away': ot2_pts_home,
                             'ot3_pts_away': ot3_pts_home,
                             'ot4_pts_away': ot4_pts_home,
                             'spread' : spread,
                             'total' : total
                            })
    
    updated_data = pd.concat([my_data, missing_data])
    updated_data.to_csv('./data/nba_betting_data_2013_present.csv')
    
update_betting_data()

updated 10 dates starting from 2021-01-30
